In [1]:
#import libraries

import pandas as pd
import pandapower as pp
from pandapower import plotting
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import math
import numba

In [2]:
#####GEOGRAPHY#####

#defining bus locations

#Mallorca
loc_eb=(3.1583316, 39.5822638)
loc_ll=(3.0399121, 39.6736644)
loc_mp=(3.092442,39.809435)
loc_so=(2.7442129,39.6002680)
loc_sr=(2.6788739, 39.6511746)
loc_va=(2.5492521, 39.5843260)
loc_cas = (2.7243168,39.5683692)
loc_sp=(2.50895716,39.53767893)

#Menorca
loc_ciut=(3.8553116,40.0033945)
loc_merc = (4.09591994,39.97708936)
loc_drag = (4.2366714,39.8912109)
loc_mahon = (4.25796449,39.89715096)

#Ibiza/Formentera generations and loads aggregated at Torrent
loc_ibiza=(1.490401,39.006430)

#Mallorca-Menorca submarine link
loc_gs=(3.42746869,39.73867947)
loc_cb = (3.83379024,39.93198792)


#defining 220kV and 132kV line lengths from qgis (km)

IB_SP=126 #https://www.ree.es/sites/default/files/01_ACTIVIDADES/Documentos/Romulo2_en.pdf

#mallorca line lengths
SP_Val=6
Val_SR1=15
Val_SR2=15
SR_SO=9
CA_SO = 4
CA_SO2 = 4
SR_LL=34
SO_LL=29
LL_MTR1=17
LL_MTR2=16
LL_EB1=15
LL_EB2=15
EB_GESA=30

#menorca line lengths
GESA_CB=41
CB_CIUT = 8
drag_mahon = 5
merc_drag = 15
ciut_drag = 36
ciut_merc = 21

In [3]:
#defining Q=P*PF

PF= 0.484322

In [4]:
#loading generation and demand for BI

df1 = pd.read_csv('Mallorca.csv')
df2 = pd.read_csv('Menorca.csv')
df3 = pd.read_csv('Ibiza.csv')

In [5]:
#creating loop every 10 minutes over 24 hours
i=0
loading_percent_sum=[['time','line','%']]
while i in range (0,144): 

    # Create empty net
    net = pp.create_empty_network()
    
    #initialize line voltages in MW assuming ibiza/formentera represented as a load bus
    hv220=220
    hv132=132

    #Mallorca buses
    b_eb_lv = pp.create_bus(net, vn_kv=hv132, name = "Es Bessons LV", geodata=loc_eb)
    b_eb_hv = pp.create_bus(net, vn_kv=hv220, name = "Es Bessons HV", geodata=loc_eb)

    b_ll = pp.create_bus(net, vn_kv=hv220, name = "LLubi", geodata=loc_ll)
    b_sr = pp.create_bus(net, vn_kv=hv220, name = "Son Rues", geodata=loc_sr)
    b_va = pp.create_bus(net, vn_kv=hv220, name = "Valldurgent", geodata=loc_va)
    b_so = pp.create_bus(net, vn_kv=hv220, name = "Son Orlandis", geodata=loc_so)
    b_mp = pp.create_bus(net, vn_kv=hv220, name = "Murterar Power", geodata=loc_mp)
    b_gs = pp.create_bus(net, vn_kv=hv132, name = "GESA", geodata=loc_gs)

    b_cas = pp.create_bus(net, vn_kv=hv220, name = "Cas Tresoare", geodata=loc_cas)

    #create buses for menorca (both modelled as load buses on 132kV network)
    b_cb = pp.create_bus(net, vn_kv=hv132, name = "Cala en Bosc", geodata=loc_cb)
    b_ciut = pp.create_bus(net, vn_kv=hv132, name = "Ciutadella", geodata=loc_ciut)
    b_mahon = pp.create_bus(net, vn_kv=hv132, name = "Mahon", geodata=loc_mahon)
    b_drag = pp.create_bus(net, vn_kv=hv132, name = "Dragonera SS", geodata=loc_drag)
    b_merc = pp.create_bus(net, vn_kv=hv132, name = "Es Merceds SS", geodata=loc_merc)

    #create bus representing ibiza/formentera (load bus on 132kV network)
    b_ibiza = pp.create_bus(net, vn_kv=hv132, name = "Elvissa PS", geodata=loc_ibiza)

    #create buses for mainland interconnector and trafo

    b_sp_lv = pp.create_bus(net, vn_kv=hv132, name = "SP HVDC LV", geodata=loc_sp) 
    b_sp_hv = pp.create_bus(net, vn_kv=hv220, name = "SP HVDC HV", geodata=loc_sp)
    
    
    #create 220 to 132kV transformers at es bessons and santa ponsa, typical capacity zotero
    trans_sp = pp.create_transformer_from_parameters(net, hv_bus=b_sp_hv, lv_bus=b_sp_lv, sn_mva=400, vn_hv_kv=220,\
                                                  vn_lv_kv=132, vkr_percent=1, vk_percent=10, pfe_kw=400.*0.01, i0_percent=1)

    trans_eb = pp.create_transformer_from_parameters(net, hv_bus=b_eb_hv, lv_bus=b_eb_lv, sn_mva=400, vn_hv_kv=220,\
                                                  vn_lv_kv=132, vkr_percent=1, vk_percent=10, pfe_kw=400*0.01, i0_percent=1)   
    
    
    #create lines and attach them to the nodes....

    #220 overhead typical properties....gull aw
    Rkm_220OH = 0.13
    Xkm_220OH = 0.425
    Cnfkm_220OH = 8.2
    MaxIkA220 = 0.8

    #132 overhead properties....hawk aw
    Rkm_132OH = 0.14
    Xkm_132OH = 0.42
    Cnfkm_132OH = 8.2
    MaxIkA132 = 0.66

    #132 Underwater/ground properties IBIZA MALLORCA
    Rkm_132UW = 0.14
    Xkm_132UW = 0.10
    Cnfkm_132UW = 200
    MaxIkA132UW = 1.6

    #132 Underwater/ground properties Mallorca Menorca
    Rkm_132UWmen = 0.14
    Xkm_132UWmen = 0.10
    Cnfkm_132UWmen = 200
    MaxIkA132men = 1.6



    #132kV underwater line properties

    line_ibiza_sp1 = pp.create_line_from_parameters(net, from_bus = b_sp_lv, to_bus = b_ibiza , length_km= IB_SP , r_ohm_per_km=Rkm_132UW, x_ohm_per_km=Xkm_132UW, \
                                                c_nf_per_km = Cnfkm_132UW, max_i_ka =MaxIkA132UW, name = 'Ibiza_SP1', geodata = [loc_ibiza, loc_sp])
    line_ibiza_sp2 = pp.create_line_from_parameters(net, from_bus = b_sp_lv, to_bus = b_ibiza, length_km= IB_SP ,  r_ohm_per_km=Rkm_132UW, x_ohm_per_km=Xkm_132UW, \
                                                c_nf_per_km = Cnfkm_132UW, max_i_ka =MaxIkA132UW , name = 'Ibiza_SP2', geodata = [loc_ibiza, loc_sp])


    #220kV network on mallorca
    line_sp_val1 = pp.create_line_from_parameters(net,  from_bus = b_sp_hv, to_bus = b_va , length_km= SP_Val , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'SP_Val', geodata = [loc_sp, loc_va])
    line_sp_val2 = pp.create_line_from_parameters(net,  from_bus = b_sp_hv, to_bus = b_va , length_km= SP_Val , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'SP_Val', geodata = [loc_sp, loc_va])

    #two lines from santa ponsa to Valldurgent 
    line_val_sr1 = pp.create_line_from_parameters(net,  from_bus = b_va, to_bus = b_sr , length_km= Val_SR1 , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH,\
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='val_sr1', geodata = [loc_va, loc_sr])
    line_val_sr2 = pp.create_line_from_parameters(net,  from_bus = b_va, to_bus = b_sr , length_km= Val_SR2 , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH,\
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'val_sr2', geodata = [loc_va, loc_sr])

    line_sr_so = pp.create_line_from_parameters(net,  from_bus = b_sr, to_bus = b_so , length_km= SR_SO , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                            c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'sr_so', geodata = [loc_sr, loc_so])
    line_cas_so1 = pp.create_line_from_parameters(net, from_bus = b_cas, to_bus = b_so, length_km= CA_SO, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'cas_so1', geodata = [loc_cas, loc_so])
    line_cas_so2 = pp.create_line_from_parameters(net, from_bus = b_cas, to_bus = b_so, length_km= CA_SO, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='cas_so2', geodata = [loc_cas, loc_so])

    line_sr_ll = pp.create_line_from_parameters(net,  from_bus = b_sr, to_bus = b_ll , length_km= SR_LL , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                            c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'sr_ll', geodata = [loc_sr, loc_ll])
    line_so_ll = pp.create_line_from_parameters(net,  from_bus = b_so, to_bus = b_ll , length_km= SO_LL , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                            c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'so_ll', geodata = [loc_so, loc_ll])


    line_LL_MP1 = pp.create_line_from_parameters(net, from_bus = b_ll, to_bus = b_mp, length_km= LL_MTR1, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                             c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='ll_mp1', geodata = [loc_ll, loc_mp])
    line_LL_MP2 = pp.create_line_from_parameters(net, from_bus = b_ll, to_bus = b_mp, length_km= LL_MTR2, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                             c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'll_mp2', geodata = [loc_ll, loc_mp])

    line_LL_EB1 = pp.create_line_from_parameters(net, from_bus = b_ll, to_bus = b_eb_hv, length_km= LL_EB1, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                             c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='ll_eb1', geodata = [loc_ll, loc_eb])
    line_LL_EB2 = pp.create_line_from_parameters(net, from_bus = b_ll, to_bus = b_eb_hv, length_km= LL_EB2, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                             c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='ll_eb2', geodata = [loc_ll, loc_eb])


    #132 lines on mallorca
    line_EB_GESA = pp.create_line_from_parameters(net, from_bus = b_eb_lv, to_bus = b_gs, length_km= EB_GESA,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                              c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name = 'eb_gesa', geodata = [loc_eb, loc_gs]) #change std type


    #132kV underwater line
    line_GESA_cb = pp.create_line_from_parameters(net, from_bus = b_gs, to_bus = b_cb, length_km= GESA_CB,  r_ohm_per_km=Rkm_132UWmen, x_ohm_per_km=Xkm_132UWmen, \
                                              c_nf_per_km = Cnfkm_132UWmen, max_i_ka =MaxIkA132men, name ='gesa_cb', geodata = [loc_gs, loc_cb]) #change std type

    #standard 132 overhead lines
    line_cb_Ciut = pp.create_line_from_parameters(net, from_bus = b_cb, to_bus = b_ciut, length_km= CB_CIUT,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                              c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='cb_cuit', geodata = [loc_cb, loc_ciut])
    line_ciut_drag = pp.create_line_from_parameters(net, from_bus = b_ciut, to_bus = b_drag, length_km= ciut_drag,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH,\
                                                c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='ciut_drag', geodata = [loc_ciut, loc_drag])
    line_ciut_merc = pp.create_line_from_parameters(net, from_bus = b_ciut, to_bus = b_merc, length_km= ciut_merc,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                                c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='ciut_merc',  geodata = [loc_ciut, loc_merc])
    line_merc_drag =  pp.create_line_from_parameters(net, from_bus = b_merc, to_bus = b_drag, length_km= merc_drag,   r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                                 c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='merc_drag', geodata = [loc_merc, loc_drag] )
    line_drag_mahon1 = pp.create_line_from_parameters(net, from_bus = b_drag, to_bus = b_mahon, length_km= drag_mahon,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH,\
                                                  c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='drag_mahon1', geodata = [loc_drag, loc_mahon])
    line_drag_mahon2 = pp.create_line_from_parameters(net, from_bus = b_drag, to_bus = b_mahon, length_km= drag_mahon,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                                  c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='drag_mahon2' , geodata = [loc_drag, loc_mahon])

    #create load buses
    #residential loads in MW for each load bus,weighted avg of population

    # 6 buses, va=, sr=, so=, ll=, eb=
    p_va=8.07/100*df1.demand[i]
    p_sr=25.29/100*df1.demand[i]    
    p_so=25.29/100*df1.demand[i]    
    p_mp=6.46/100*df1.demand[i]    
    p_ll=13.73/100*df1.demand[i]    
    p_eb=21.15/100*df1.demand[i]    
    

    #menorca load at 1320 is 53MW
    p_ciut=37.06/100*df2.demand[i]
    p_cb = 16.6/100*df2.demand[i]
    p_mahon= 46.33/100*df2.demand[i]

    #ibiza and formentera combined load minus generation at 1320. 
    p_ibiza=df3.demand[i]

    L_eb = pp.create_load(net, bus=b_eb_lv, p_mw=p_eb, q_mvar=p_eb*PF, name = "Load Es Bessons")
    L_ll = pp.create_load(net, bus=b_ll, p_mw=p_ll, q_mvar=p_ll*PF, name = "Load LLubi")
    L_sr = pp.create_load(net, bus=b_sr, p_mw=p_sr, q_mvar=p_sr*PF, name = "Load Son Reus")
    L_va = pp.create_load(net, bus=b_va, p_mw=p_va, q_mvar=p_va*PF, name = "Load Valldurgent")
    L_so = pp.create_load(net, bus=b_so, p_mw=p_so, q_mvar=p_so*PF, name = "Load Son Orlandis")
    L_mp = pp.create_load(net, bus=b_mp, p_mw=p_mp, q_mvar=p_mp*PF, name = "Load Murtretar") 

    L_cb = pp.create_load(net, bus=b_cb, p_mw=p_cb, q_mvar=p_cb*PF, name = "Load Cala en bosc")
    L_ciut = pp.create_load(net, bus=b_ciut, p_mw=p_ciut, q_mvar=p_ciut*PF, name = "Load ciutadella")
    L_mahon = pp.create_load(net, bus=b_drag, p_mw=p_mahon, q_mvar=p_mahon*PF, name = "Load mahon")

    L_ibiza = pp.create_load(net, bus=b_ibiza, p_mw=p_ibiza, q_mvar=p_ibiza*PF, name = "Load Ibiza/Formentera")

    L_sp_react = pp.create_load(net, bus=b_sp_lv, p_mw=0, q_mvar=237, name = "Reactor")

    
    P_sr_net=0
    P_cas_net=0
    P_MDG_net=0
    P_ib_net=0
    
    #define generation Mallorca

    P_sr = (df1.cc[i])/2+df1.others[i]+df1.waste[i]+df1.cogen[i]
    P_cas = (df1.cc[i])/2+df1.pv[i] #pvs are located mostly south, nearest to cas tresorer
    P_pv_M = (df1.newsolar[i]/1600*1050)

    #define Menorca generation 32.6MW DIESEL 21.8MW GT (.1 wind, 1.1 solar, all into grid at mahon)
    P_MDG = df2.gen[i]
    P_pv_m = df2.newsolar[i]/170*175

    #define Ibiza & Formentera generation
    P_ib = df3.gen[i]
    P_pv_I = (df3.newsolar[i]/420*750)
    
    #defining plant shut off order in Mallorca, cas first
    if P_pv_M<P_cas:
        P_sr_net=P_sr
        P_cas_net=P_cas-P_pv_M   
    elif P_pv_M>P_cas and P_pv_M<P_sr:
        P_sr_net=P_sr-(P_pv_M-P_cas)
        P_cas_net=0    
    elif P_pv_M>P_sr:
        P_sr_net=0
        P_cas_net=0

    #reducing oil plant for PV in Menorca
    if P_pv_m<P_MDG:
        P_MDG_net=P_MDG-P_pv_m
    elif P_pv_m>P_ib:
        P_MDG_net=0
        
    #reducing oil plant for PV in Ibiza
    if P_pv_I<P_ib:
        P_ib_net=P_ib-P_pv_I
    elif P_pv_I>P_ib:
        P_ib_net=0
        
        
    #create generator buses

    gen_sr = pp.create_gen(net, bus = b_sr, p_mw = P_sr_net, vm_pu = 1.05 , name = "Son Reus GT and Waste")
    gen_cas = pp.create_gen(net, bus = b_cas, p_mw = P_cas_net, vm_pu = 1.05, name = "Cas Tresoer Gas Turbine")
    #gen_mahon = pp.create_gen(net, bus = b_mahon, p_mw = P_MDG_net, vm_pu = 1.05, name = 'Mahon Diesel Gen')
    gen_ib = pp.create_gen(net, bus = b_ibiza, p_mw = P_ib_net, vm_pu = 1.05, name = 'Ibiza Gen')

    
    #PV generation
    gen_ib_pv = pp.create_gen(net, bus = b_ibiza, p_mw = P_pv_I, vm_pu = 1.05, name = 'Torrent PV rooftop')
    gen_drag_pv = pp.create_gen(net, bus = b_drag, p_mw = P_pv_m, vm_pu = 1.05, name = 'Dragonera PV rooftop')
    
    gen_sr_pv = pp.create_gen(net, bus = b_sr, p_mw = P_pv_M/6, vm_pu = 1.05, name = 'SR PV rooftop')
    gen_so_pv = pp.create_gen(net, bus = b_so, p_mw = P_pv_M/6, vm_pu = 1.05, name = 'SO PV rooftop')
    gen_eb_pv = pp.create_gen(net, bus = b_eb_hv, p_mw = P_pv_M/6, vm_pu = 1.05, name = 'EB PV rooftop')
    gen_ll_pv = pp.create_gen(net, bus = b_ll, p_mw = P_pv_M/6, vm_pu = 1.05, name = 'LL PV rooftop')
    gen_va_pv = pp.create_gen(net, bus = b_va, p_mw = P_pv_M/6, vm_pu = 1.05, name = 'EB PV rooftop')
    gen_mp_pv = pp.create_gen(net, bus = b_mp, p_mw = P_pv_M/6, vm_pu = 1.05, name = 'EB PV rooftop')
    

    #create swing bus to represent HVDC cable

    swing_hvdc = pp.create_ext_grid(net, bus=b_sp_hv, vm_pu=1.05, name="Cometa Swing Bus")


    pp.runpp(net)
    
    #print(net.res_line.loading_percent)
    j=0
    for j in range(0,23):
        if net.res_line.loading_percent[j] > 0:
            loading_percent_sum.append([i/6,j,net.res_line.loading_percent[j]])

        j+=1
    
    #calc average of line loads    
    #j=0
    #line_sum=0
    #for j in range(0,23):
    #    line_sum+=net.res_line.loading_percent[j]
    #    j+=1
    #loading_percent_sum.append([i/6,j,line_sum/23])
    
    i+=1
    


In [6]:
#print(loading_percent_sum)
## convert your array into a dataframe
out = pd.DataFrame (loading_percent_sum)

## save to xlsx file

filepath = 'out.xlsx'

out.to_excel(filepath, index=False)

In [7]:
net.line.head(24)

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,Ibiza_SP1,None,15,14,126.0,0.14,0.100,200.0,0.0,1.60,1.0,1,None,True
1,Ibiza_SP2,None,15,14,126.0,0.14,0.100,200.0,0.0,1.60,1.0,1,None,True
2,SP_Val,None,16,4,6.0,0.13,0.425,8.2,0.0,0.80,1.0,1,None,True
3,SP_Val,None,16,4,6.0,0.13,0.425,8.2,0.0,0.80,1.0,1,None,True
4,val_sr1,None,4,3,15.0,0.13,0.425,8.2,0.0,0.80,1.0,1,None,True
5,val_sr2,None,4,3,15.0,0.13,0.425,8.2,0.0,0.80,1.0,1,None,True
6,sr_so,None,3,5,9.0,0.13,0.425,8.2,0.0,0.80,1.0,1,None,True
7,cas_so1,None,8,5,4.0,0.13,0.425,8.2,0.0,0.80,1.0,1,None,True
8,cas_so2,None,8,5,4.0,0.13,0.425,8.2,0.0,0.80,1.0,1,None,True
9,sr_ll,None,3,2,34.0,0.13,0.425,8.2,0.0,0.80,1.0,1,None,True
